In [ ]:
import pyspark
from delta import *
from delta.tables import *
from pyspark.sql.functions import *

In [ ]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

In [ ]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
deltaTable = DeltaTable.forPath(spark, "./out/delta-table")

In [ ]:
deltaTable

In [ ]:
deltaTable.toDF().show()

In [ ]:
# Update every even value by adding 100 to it
deltaTable.update(condition = expr("id % 2 == 0"), set = { "id": expr("id + 100") })

In [ ]:
deltaTable.toDF().show()

At this point, visit http://localhost:4040 to observe Spark jobs and workloads there.

In [ ]:
# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

In [ ]:
deltaTable.toDF().show()

In [ ]:
# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

In [ ]:
deltaTable.toDF().show()

In [ ]:
# Read older versions of data using time travel
df = spark.read.format("delta").option("versionAsOf", 0).load("./out/delta-table")
df.show()

In [ ]:
# Write a stream of data to a table
streamingDf = spark.readStream.format("rate").load()

stream = streamingDf.selectExpr("value as id") \
  .writeStream.format("delta") \
  .option("checkpointLocation", "./out/checkpoint") \
  .start("./out/delta-table")

In [ ]:
deltaTable.toDF().show()

In [ ]:
stream.stop()

In [ ]:
deltaTable.toDF().count()

In [ ]:
deltaTable.toDF().head(5)

In [ ]:
deltaTable.toDF().tail(5)

In [ ]:
deltaTable.toDF().orderBy("id", ascending=False).collect()

In [ ]:
# Working with Spark SQL
# https://spark.apache.org/docs/3.1.1/api/python/getting_started/quickstart.html#Working-with-SQL

df = deltaTable.toDF()
df.createOrReplaceTempView("tableA")

In [ ]:
spark.sql("SELECT count(*) FROM tableA").show()

In [ ]:
spark.sql("DESCRIBE tableA").show()

In [ ]:
spark.sql("SELECT * FROM tableA ORDER BY id DESC").show()